In [1]:
# imports
%reload_ext autoreload
%autoreload 2
%matplotlib inline 

from matplotlib import pyplot as plt

from utils.basic_utils import *
from utils.pricing import *
import utils.fundamental as fu

pd.options.display.float_format = '{:,.2f}'.format

Loading utils/config.json


In [2]:
# use the latest saved data for profile and quote info
dates = read_dates('quote')
tgt_date = dates[-1] # last date saved in S3

quotes = load_csvs('quote_consol', [tgt_date])
quotes.set_index('symbol', drop=False, inplace=True)

profile = load_csvs('summary_detail', ['assetProfile'])
profile.set_index('symbol', drop=False, inplace=True)

keystats = load_csvs('summary_detail', ['defaultKeyStatistics/' + str(tgt_date)])
keystats.set_index('symbol', drop=False, inplace=True)
finstats = load_csvs('summary_detail', ['financialData/' + str(tgt_date)])
finstats.set_index('symbol', drop=False, inplace=True)

Loading file quote/csv/2019-06-04
Loading file summary-categories/assetProfile
Loading file summary-categories/defaultKeyStatistics/2019-06-04
Loading file summary-categories/financialData/2019-06-04


In [3]:
# read from S3
val_df = pd.read_csv(csv_load(f'valuation/waterfall/{tgt_date}'), parse_dates=True)
val_df.storeDate = pd.to_datetime(val_df.storeDate, unit='s')
val_df.set_index(['storeDate', 'symbol'], inplace=True)
val_df.dropna(subset=['premDisc'], inplace=True)
tickers = val_df.reset_index().symbol

In [4]:
val_df.columns

Index(['AAT', 'AT', 'FX', 'ROA', 'ROE', 'baseMult', 'baseRate', 'currentValue',
       'discountRate', 'equityRiskPremium', 'equityValue', 'financialCurrency',
       'growthMult', 'growthRate', 'netDebt', 'npvGrowth', 'npvGrowthCF',
       'npvSteadyCF', 'premDisc', 'projCashROE', 'projFedFundsIncr',
       'steadyCF', 'totalReinvCapital'],
      dtype='object')

In [5]:
q_cols = ['forwardPE', 'trailingPE', 'marketCap', 'regularMarketPrice']
p_cols = ['sector', 'industry', 'country']
k_cols = ['pegRatio', 'shortPercentOfFloat']
f_cols = ['earningsGrowth', 'recommendationMean', 'targetMeanPrice', 'targetMedianPrice', 'numberOfAnalystOpinions']
for c in p_cols: val_df.loc[:, c] = tickers.map(profile[c].to_dict()).values
for c in q_cols: val_df.loc[:, c] = tickers.map(quotes[c].to_dict()).values
for c in k_cols: val_df.loc[:, c] = tickers.map(keystats[c].to_dict()).values
for c in f_cols: val_df.loc[:, c] = tickers.map(finstats[c].to_dict()).values

In [6]:
large_vals = [
    'netDebt', 'npvSteadyCF', 'equityValue', 
    'currentValue', 'totalReinvCapital', 'steadyCF', 'projCashROE',
    'npvGrowth', 'npvGrowthCF', 'projCashROE', 'marketCap']
# convert large values to billions
val_df.loc[:, large_vals] = val_df.loc[:, large_vals] / 10**9

# high level pre-processing / clean up
val_df.loc[:, 'pegRatio'] = val_df.forwardPE / (val_df.growthRate * 100)
div_cols = ['targetMeanPrice', 'targetMedianPrice']
val_df.loc[:, div_cols] = val_df[div_cols].div(
    tickers.map(
        quotes.regularMarketPrice.to_dict()).values, axis=0)
val_df.replace([np.inf, -np.inf], np.nan, inplace=True)
val_df.shape

(1099, 37)

In [7]:
# remove outliers from universe
show = ['premDisc', 'pegRatio', 'forwardPE', 'growthRate', 
        'shortPercentOfFloat', 'targetMedianPrice']
treshold = 3
no_out_df = fu.excl_outliers(val_df, show, treshold)
no_out_df.shape

(1000, 37)

In [8]:
# mask results by valuation
mask_on = True
mask = (val_df.premDisc > 0) & (val_df.premDisc < 3)
clean_df = no_out_df.loc[mask].copy() if mask_on else no_out_df.copy()
clean_df.shape

(614, 37)

In [9]:
# one company sampling
ticker = 'AAPL'
print(ticker in tickers.values, ticker in clean_df.index.levels[1])
val_df.loc[(slice(None),ticker),:].T

True True


storeDate,2019-06-04
symbol,AAPL
AAT,1.25
AT,0.69
FX,1.00
ROA,0.17
ROE,0.30
baseMult,13.09
baseRate,0.02
currentValue,821.66
discountRate,0.08


In [10]:
# aggregate / deep dive metrics
gby = ['sector',]
gby = ['sector', 'industry', 'symbol'] # by uncommenting this line deep dive into sectors below
zoom_in = [x for x in clean_df.sector.unique()] # all sectors
zoom_in = ['Healthcare'] # or just a few selected
agg_functions = 'median'
sort_by = ['premDisc']

ind_sum_df = clean_df.groupby(by=gby).agg(agg_functions)[show]
if 'symbol' in gby: ind_sum_df = ind_sum_df.loc[(zoom_in,),:]
else: ind_sum_df = ind_sum_df
ind_sum_df.sort_values(by=sort_by)

premDisc  \
sector     industry                                 symbol             
Healthcare Biotechnology                            EXEL        0.26   
           Drug Manufacturers - Specialty & Generic SUPN        0.54   
           Drug Manufacturers - Major               BIIB        0.57   
           Biotechnology                            UTHR        0.63   
           Medical Distribution                     MCK         0.65   
                                                    ABC         0.67   
                                                    CAH         0.75   
           Drug Manufacturers - Major               ABBV        0.89   
                                                    BMY         0.93   
                                                    GILD        0.97   
                                                    AMGN        1.03   
           Health Care Plans                        CI          1.18   
           Medical Care                             UHS         1.31   
           Biotechnology                            CBM         1.35   
           Medical Care                             EHC         1.36   
           Medical Devices                          ALGN        1.38   
           Diagnostics & Research                   BIO         1.39   
                                                    DGX         1.45   
           Medical Care                             CHE         1.46   
           Medical Devices                          GMED        1.46   
           Medical Care                             AMN         1.47   
           Medical Distribution                     HSIC        1.52   
           Health Care Plans                        ANTM        1.53   
                                                    UNH         1.66   
           Medical Care                             HCA         1.69   
                                                    AMED        1.74   
           Diagnostics & Research                   A           1.83   
           Drug Manufacturers - Major               JNJ         1.89   
           Medical Distribution                     PDCO        1.91   
           Health Care Plans                        CNC         1.93   
           Drug Manufacturers - Major               NVS         2.02   
           Biotechnology                            INCY        2.06   
           Medical Care                             MD          2.07   
           Drug Manufacturers - Major               GSK         2.08   
           Diagnostics & Research                   WAT         2.11   
           Medical Instruments & Supplies           MASI        2.13   
           Drug Manufacturers - Major               BAYRY       2.21   
           Medical Instruments & Supplies           STE         2.24   
                                                    ISRG        2.26   
           Diagnostics & Research                   DXCM        2.39   
           Drug Manufacturers - Major               PFE         2.39   
           Medical Instruments & Supplies           COO         2.43   
           Drug Manufacturers - Major               LLY         2.48   
           Diagnostics & Research                   MTD         2.54   
           Drug Manufacturers - Major               MRK         2.58   
           Long-Term Care Facilities                ENSG        2.70   
           Medical Instruments & Supplies           ICUI        2.70   
                                                    VAR         2.82   
           Diagnostics & Research                   ILMN        2.95   

                                                            pegRatio  \
sector     industry                                 symbol             
Healthcare Biotechnology                            EXEL        0.33   
           Drug Manufacturers - Specialty & Generic SUPN        0.32   
           Drug Manufacturers - Major               BIIB        0.65   
           Biotechnology   

In [ ]:
# rank results relative to sector or entire universe
low_better = ['premDisc', 'pegRatio', 'forwardPE', 'shortPercentOfFloat']
high_better = ['earningsGrowth', 'targetMedianPrice']
cols = low_better + high_better

by_sector_on = True

rank_df = clean_df.copy()
if by_sector_on:
    # relative to sector
    super_list = []
    for s in rank_df.sector.unique():
        group = rank_df.loc[rank_df.sector == s, cols]
        ranked_df = fu.rank_group(group, low_better, high_better)
        super_list.append(ranked_df)
    ranked_df = pd.concat(super_list, axis=0)
else:
    # relative to market
    ranked_df = fu.rank_group(rank_df, low_better, high_better)

ranked_df.loc[:, 'eq_wgt_mean_rank'] = ranked_df.mean(axis=1)
ranked_df.sort_values('eq_wgt_mean_rank')